In [14]:
%%capture
!pip install xgboost
!pip install hummingbird-ml

In [2]:
import pickle
from os import listdir
from os.path import isfile, join
import numpy as np
import xgboost as xgb

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
with open('drive/MyDrive/RealLifeViolencePickle/training_violence2.pickle', 'rb') as handle:
    training_run2 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/training_violence3.pickle', 'rb') as handle:
    training_run3 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/training_violence4.pickle', 'rb') as handle:
    training_run4 = pickle.load(handle)
with open('drive/MyDrive/RealLifeViolencePickle/valid_violence.pickle', 'rb') as handle:
    allTestingData = pickle.load(handle)

In [5]:
allTrainingData = np.concatenate([training_run2, training_run3, training_run4])
allTrainingData.shape

(73384, 3780)

In [6]:
allTestingData.shape

(18508, 3780)

In [7]:
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence2.pickle', 'rb') as labels:
  labels_run2 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence3.pickle', 'rb') as labels:
  labels_run3 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence4.pickle', 'rb') as labels:
  labels_run4 = pickle.load(labels)
with open('drive/MyDrive/RealLifeViolencePickle/labels_violence_valid.pickle', 'rb') as labels:
  listOfFrameLabelsTest = pickle.load(labels)

In [8]:
listOfFrameLabels = np.concatenate([labels_run2, labels_run3, labels_run4])

In [9]:
framesTrain, labelsTrain = allTrainingData, listOfFrameLabels
framesTest, labelsTest = allTestingData, listOfFrameLabelsTest

In [10]:
import xgboost as xgb
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score

xgb_model = xgb.XGBClassifier()
xgb_model.fit(training_run2, labels_run2)

retrained_model = xgb.XGBClassifier(objective='binary:logistic')
retrained_model.fit(training_run3, labels_run3, xgb_model = xgb_model.get_booster())
labelPredict = retrained_model.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('ROC-AUC Score : ', roc_auc_score(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

[[8692    0]
 [1533 8283]]
              precision    recall  f1-score   support

           0       0.85      1.00      0.92      8692
           1       1.00      0.84      0.92      9816

    accuracy                           0.92     18508
   macro avg       0.93      0.92      0.92     18508
weighted avg       0.93      0.92      0.92     18508

ROC-AUC Score :  0.9219132029339854
Total Accuracy:  0.9171709531013615


In [12]:
final_model = xgb.XGBClassifier(objective='binary:logistic')
final_model.fit(training_run4, labels_run4, xgb_model = retrained_model.get_booster())
labelPredict = final_model.predict(framesTest)
print(confusion_matrix(labelsTest, labelPredict))
print(classification_report(labelsTest, labelPredict))
print('ROC-AUC Score : ', roc_auc_score(labelsTest, labelPredict))
print('Total Accuracy: ',accuracy_score(labelsTest,labelPredict))

[[8207  485]
 [ 517 9299]]
              precision    recall  f1-score   support

           0       0.94      0.94      0.94      8692
           1       0.95      0.95      0.95      9816

    accuracy                           0.95     18508
   macro avg       0.95      0.95      0.95     18508
weighted avg       0.95      0.95      0.95     18508

ROC-AUC Score :  0.9457662265013571
Total Accuracy:  0.9458612491895396


In [13]:
with open('drive/MyDrive/EventDetection/saved_models/skl_xgboost.pickle', 'wb')as saved:
    pickle.dump(final_model, saved, protocol=pickle.HIGHEST_PROTOCOL)

In [16]:
from hummingbird.ml import convert,load
# Use Hummingbird to convert the model to PyTorch
model = convert(final_model, 'pytorch')
# Save the model
model.save('drive/MyDrive/EventDetection/saved_models/pytorch_xgboost')